In [13]:
import pandas as pd
import joblib
import os
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV

In [14]:
	
def metrica_performance(model,x_train,x_test,y_train,y_test):
    """
    Função que calcula métricas de perfomance para o modelo.
    """
    y_pred_treino = model.predict(x_train)
    y_pred_teste = model.predict(x_test)

    #R²
    r2_treino = r2_score(y_train, y_pred_treino)
    r2_teste = r2_score(y_test, y_pred_teste)

    #mae
    mae_treino = mean_absolute_error(y_train, y_pred_treino)
    mae_teste = mean_absolute_error(y_test, y_pred_teste)
    
    #rmse
    rmse_treino = root_mean_squared_error(y_train, y_pred_treino)
    rmse_teste = root_mean_squared_error(y_test, y_pred_teste)
    
    #Concatentando informações
    resultado = {'Métrica': ['R²', 'MAE', 'RMSE'],
                 'Treino': [r2_treino, mae_treino, rmse_treino],
                 'Teste': [r2_teste, mae_teste, rmse_teste] }

    df_resultado = pd.DataFrame(resultado)
    return round(df_resultado,3)

In [15]:
# Carregar base de dados
base_dados = pd.read_csv("imoveis_belo_horizonte_ML_distancias.csv", sep=";", decimal=".")

# remove espaços extras e pontos finais perdidos
base_dados = base_dados.applymap(lambda x: str(x).strip().rstrip('.') if isinstance(x, str) else x)

# Remover colunas não utilizadas
base_dados_ML = base_dados.drop(
    ["CEP_opencage", "latitude", "longitude", "bairro_tratada", "endereco", "endereco_norm", "bairro"], axis=1)

/var/folders/sj/grpf2mg12b3813fqgkd99x3c0000gn/T/ipykernel_18506/513182060.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  base_dados = base_dados.applymap(lambda x: str(x).strip().rstrip('.') if isinstance(x, str) else x)


In [16]:
base_dados_ML

,preco,tamanho (m²),qtd_quartos,qtd_banheiros,qtd_vagas_garagem,bairro_mais_barato,bairro_norm,dist_praca_da_liberdade_km,dist_lagoa_da_pampulha_km,dist_parque_americo_renne_km,dist_parque_das_mangabeiras_km,dist_circuito_da_liberdade_km,dist_mercado_central_km,dist_igreja_sao_jose_km
0,1450000.0,135.0,4,2.0,2,0,gutierrez,2.188520,8.801619,2.714656,5.864126,2.188520,2.810012,2.714656
1,1070000.0,115.0,2,2.0,2,0,santo antonio,1.583653,9.987610,2.969234,4.932790,1.583653,3.107370,2.969234
2,280000.0,60.0,3,1.0,1,0,estoril,4.159618,10.829141,5.316524,7.242725,4.159618,5.430932,5.316524
3,1335000.0,78.0,3,2.0,2,0,sao pedro,1.064588,10.341657,2.797019,4.143447,1.064588,2.948678,2.797019
4,530000.0,75.0,3,2.0,2,0,castelo,9.299561,3.348703,8.129909,12.696133,9.299561,8.043440,8.129909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6879,330000.0,54.0,2,1.0,1,0,juliana,13.270785,5.881498,11.423627,14.866806,13.270785,11.268800,11.423627
6880,715000.0,71.0,3,2.0,2,0,padre eustaquio,5.949846,6.637607,5.525873,9.636536,5.949846,5.517059,5.525873
6881,700000.0,140.0,4,3.0,2,0,alto caicaras,1.904097,7.913507,0.469065,4.947099,1.904097,0.495049,0.469065
6882,390000.0,49.0,2,2.0,1,0,gloria,8.750730,5.839315,8.062123,12.398827,8.750730,8.021085,8.062123


In [17]:
# Criar variáveis dummies para o bairro
df_ML_ready = pd.get_dummies(
    base_dados_ML,
    columns=['bairro_norm'],
    prefix='bairro',
    drop_first=True,
    dtype=int
)

# Definição da variável target
y = "preco"
y_rlm = df_ML_ready[y]

# Features
x_drop = "preco"
x_rlm = df_ML_ready.drop(x_drop, axis=1)


In [18]:
#Salvando em arquivo CSV
nome_do_arquivo = "imoveis_zap_belo_horizonte_ML_ready.csv"
df_ML_ready.to_csv(nome_do_arquivo, 
                   index=False, 
                   sep=";", 
                   encoding="utf-8-sig")

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x_rlm, y_rlm,
                                                     test_size=0.3,
                                                       random_state=42)

In [20]:
x_rlm = df_ML_ready.drop(columns=['preco']).select_dtypes(include=['int64', 'float64'])
y_rlm = df_ML_ready['preco'].astype(float)

In [21]:
model_columns = list(x_train.columns)
os.makedirs("joblib", exist_ok=True)
joblib.dump(model_columns, os.path.join("joblib", "model_columns.joblib"))

['joblib/model_columns.joblib']

# *3.1. Regressão Linear Múltipla*

* Visando Compreender o peso de cada variável na composição do custo do imóvel, inicialmente trabalharemos com a Reg.Linear Múltipla
* Statsmodels para interpretação dos coeficientes e comportamento da regressão
* LinearRegression para avaliar performance e geneleralização da regressão

In [22]:
# Substituir infinitos por NaN
x_rlm = x_rlm.replace([np.inf, -np.inf], np.nan)

# Remover linhas com NaN em X ou Y
dados = pd.concat([x_rlm, y_rlm], axis=1).dropna()

# Recriar X e Y limpos
x_rlm = dados.drop(columns=["preco"])
y_rlm = dados["preco"]

# Garantir que todas as colunas sejam numéricas
x_rlm = x_rlm.astype(float)
y_rlm = y_rlm.astype(float)

# Treinar modelo
modelo_rlm = sm.OLS(y_rlm, sm.add_constant(x_rlm)).fit()

In [23]:
#Alimentando o modelo LinearRegression
model_lr = LinearRegression()
model_lr.fit(x_train, y_train)

LinearRegression()

In [24]:
# Calculando as métricas de performance
resultado_lr = metrica_performance(model_lr,x_train,x_test,y_train,y_test)

In [25]:
resultado_lr

,Métrica,Treino,Teste
0,R²,0.742,0.720
1,MAE,269694.382,277373.687
2,RMSE,387332.817,397960.556


### *3.1.1 Avaliando Modelo*

In [26]:
#Visão geral
print(modelo_rlm.summary())

                            OLS Regression Results                            
Dep. Variable:                  preco   R-squared:                       0.738
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     98.09
Date:                Sun, 28 Sep 2025   Prob (F-statistic):               0.00
Time:                        10:56:18   Log-Likelihood:                -98370.
No. Observations:                6884   AIC:                         1.971e+05
Df Residuals:                    6691   BIC:                         1.984e+05
Df Model:                         192                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

## *3.2 Random Forest Regressor*
* O Random Forest Regressor é um tipo de ensemble baseado em um método chamado Bagging.

In [27]:
#Criando modelo
rfr = RandomForestRegressor(random_state=42,n_jobs=-1)

In [28]:
#Estimando parametros através do Grid
param_grid = {
    'n_estimators': [200, 300,500],  
    'max_depth': [15, 20, 25, 30],        
    'min_samples_split': [3, 4, 5, 10],       
    'min_samples_leaf': [2, 4, 6],                  
}

grid_search = GridSearchCV(
    estimator=rfr, 
    param_grid=param_grid, 
    cv=3, 
    scoring='neg_mean_absolute_error',
    verbose=2, 
    n_jobs=-1
)
grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=3, n_estimators=200; total time=   4.3s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=3, n_estimators=200; total time=   4.3s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=3, n_estimators=200; total time=   4.4s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=3, n_estimators=300; total time=   6.7s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=3, n_estimators=300; total time=   6.8s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=3, n_estimators=300; total time=   6.8s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=4, n_estimators=200; total time=   4.6s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=4, n_estimators=200; total time=   4.7s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=  11.2s
[CV] END max_depth=15, min_s

GridSearchCV(cv=3, estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [15, 20, 25, 30],
                         'min_samples_leaf': [2, 4, 6],
                         'min_samples_split': [3, 4, 5, 10],
                         'n_estimators': [200, 300, 500]},
             scoring='neg_mean_absolute_error', verbose=2)

In [29]:
#Avaliar melhor resultado
print(grid_search.best_params_)
model_rfr = grid_search.best_estimator_

{'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 500}


In [30]:
# Calculando as métricas de performance
resultado_rfr = metrica_performance(model_rfr,x_train,x_test,y_train,y_test)

In [31]:
resultado_rfr

,Métrica,Treino,Teste
0,R²,0.950,0.803
1,MAE,100626.542,201132.373
2,RMSE,171334.450,333213.391


## *3.3. XGBoost (eXtreme Gradient Boosting)*
* O Gradient Boosting é um tipo de ensemble baseado em um método chamado Boosting.

In [32]:
#Criando Modelo
xgb = XGBRegressor(random_state=42,
                    n_jobs=-1)

In [33]:
#Estimando parametros através do Grid
param_grid_xgb = {
    'n_estimators': [300,400,500,600],             
    'learning_rate': [0.01,0.02,0.05],           
    'max_depth': [10,15,20],                    
    'subsample': [0.5,0.6,0.7,0.8]                 
}

grid_search_xgb = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid_xgb,
    cv=3, 
    scoring='neg_mean_absolute_error',
    verbose=4,
    n_jobs=-1
)
grid_search_xgb.fit(x_train, y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV 3/3] END learning_rate=0.01, max_depth=10, n_estimators=300, subsample=0.5;, score=-209452.544 total time=   5.0s
[CV 1/3] END learning_rate=0.01, max_depth=10, n_estimators=300, subsample=0.5;, score=-219908.137 total time=   5.0s
[CV 2/3] END learning_rate=0.01, max_depth=10, n_estimators=300, subsample=0.5;, score=-215246.157 total time=   5.2s
[CV 3/3] END learning_rate=0.01, max_depth=10, n_estimators=300, subsample=0.6;, score=-208341.786 total time=   5.2s
[CV 2/3] END learning_rate=0.01, max_depth=10, n_estimators=300, subsample=0.6;, score=-214695.289 total time=   5.3s
[CV 1/3] END learning_rate=0.01, max_depth=10, n_estimators=300, subsample=0.6;, score=-219397.257 total time=   5.4s
[CV 1/3] END learning_rate=0.01, max_depth=10, n_estimators=300, subsample=0.7;, score=-219403.325 total time=   5.7s
[CV 2/3] END learning_rate=0.01, max_depth=10, n_estimators=300, subsample=0.7;, score=-214883.949 total time= 

GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=-1, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.02, 0.05],
                         'max_depth': [10, 15, 20],
                         'n_estimators': [300, 400, 500, 600],
                         'subsample': [0.5, 0.6, 0.7, 0.8]},
             scoring='neg_mean_absolute_error', verbose=4)

In [34]:
#Consultando melhor resultado
print(grid_search_xgb.best_params_)
model_xgb = grid_search_xgb.best_estimator_

{'learning_rate': 0.02, 'max_depth': 15, 'n_estimators': 400, 'subsample': 0.5}


In [35]:
# Calculando as métricas de performance
resultado_xgb = metrica_performance(model_xgb,x_train,x_test,y_train,y_test)

In [36]:
resultado_xgb

,Métrica,Treino,Teste
0,R²,0.989,0.822
1,MAE,54590.703,191111.214
2,RMSE,79161.197,317432.187


# *3.4. Rede Neural Artificial*
* *Etapa adicional para escalonamento dos dados*

In [37]:
#Escalonamento dos dados
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [38]:
#Definição inicial do modelo
mlp = MLPRegressor(max_iter=1000, early_stopping=True, random_state=42)

In [39]:
#Etapa de tunnning 
param_grid_mlp = {
    'hidden_layer_sizes': [(50, 25), (100, 50), (100,)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01]
}

grid_search_mlp = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid_mlp,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=2,
    n_jobs=-1
)

grid_search_mlp.fit(x_train_scaled, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(50, 25), learning_rate_init=0.01; total time=   4.2s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(50, 25), learning_rate_init=0.01; total time=   6.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(50, 25), learning_rate_init=0.01; total time=   6.7s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(100, 50), learning_rate_init=0.01; total time=   5.8s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(100, 50), learning_rate_init=0.01; total time=   8.2s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(100,), learning_rate_init=0.001; total time=   0.6s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(100,), learning_rate_init=0.001; total time=   0.5s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(100,), learning_rate_init=0.001; total time=   0.5s
[CV] END activation=relu, alpha=0.0

/Users/wirysmendes/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(50, 25), learning_rate_init=0.001; total time=  29.8s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(50, 25), learning_rate_init=0.001; total time=  30.3s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(50, 25), learning_rate_init=0.01; total time=   8.7s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(100,), learning_rate_init=0.01; total time=  17.2s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(100, 50), learning_rate_init=0.001; total time=  37.6s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(100,), learning_rate_init=0.01; total time=  25.6s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(50, 25), learning_rate_init=0.01; total time=   3.2s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(50, 25), learning_rate_init=0.01; total time=   8.9s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(100,), learning_rate_init=0.01; total t

/Users/wirysmendes/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(50, 25), learning_rate_init=0.001; total time=  39.8s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100, 50), learning_rate_init=0.001; total time=  36.7s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100,), learning_rate_init=0.01; total time=  18.3s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100, 50), learning_rate_init=0.001; total time=  41.8s


/Users/wirysmendes/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.01, hidden_layer_sizes=(50, 25), learning_rate_init=0.001; total time=  22.3s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100,), learning_rate_init=0.01; total time=  24.9s
[CV] END activation=relu, alpha=0.01, hidden_layer_sizes=(50, 25), learning_rate_init=0.01; total time=   7.7s
[CV] END activation=relu, alpha=0.01, hidden_layer_sizes=(50, 25), learning_rate_init=0.01; total time=   8.3s
[CV] END activation=relu, alpha=0.01, hidden_layer_sizes=(50, 25), learning_rate_init=0.01; total time=   5.4s
[CV] END activation=relu, alpha=0.01, hidden_layer_sizes=(100, 50), learning_rate_init=0.01; total time=   5.5s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100,), learning_rate_init=0.01; total time=  33.3s
[CV] END activation=relu, alpha=0.01, hidden_layer_sizes=(100,), learning_rate_init=0.001; total time=   0.5s
[CV] END activation=relu, alpha=0.01, hidden_layer_sizes=(100,), learning_rate_init=0.001; total time=   0.4s
[CV

GridSearchCV(cv=3,
             estimator=MLPRegressor(early_stopping=True, max_iter=1000,
                                    random_state=42),
             n_jobs=-1,
             param_grid={'activation': ['relu', 'tanh'],
                         'alpha': [0.0001, 0.001, 0.01],
                         'hidden_layer_sizes': [(50, 25), (100, 50), (100,)],
                         'learning_rate_init': [0.001, 0.01]},
             scoring='neg_mean_absolute_error', verbose=2)

In [40]:
# Consultando melhor configuração
print(grid_search_mlp.best_params_)
model_rna = grid_search_mlp.best_estimator_

{'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (50, 25), 'learning_rate_init': 0.01}


In [41]:
# Calculando as métricas de performance
resultado_rna = metrica_performance(model_rna,x_train_scaled,x_test_scaled,y_train,y_test)

In [42]:
resultado_rna

,Métrica,Treino,Teste
0,R²,0.783,0.757
1,MAE,233633.221,244526.676
2,RMSE,355160.991,370491.436


# *4. Agregando resultados para comparação*

In [43]:
resultado_xgb["Modelo"] = "XGBRegressor"
resultado_rfr["Modelo"] = "RandomForestRegressor"
resultado_rna["Modelo"] = "MLPRegressor"
resultado_lr["Modelo"] = "LinearRegression"

In [44]:
# Juntando todos os resultados
resultado = pd.concat([resultado_lr,resultado_rfr,resultado_xgb,resultado_rna],ignore_index=True)

In [45]:
resultado_pivot = resultado.pivot(index='Modelo', columns='Métrica', values=['Treino', 'Teste'])
round(resultado_pivot,2)

Treino                       Teste                 
Métrica                      MAE       RMSE    R²        MAE       RMSE    R²
Modelo                                                                       
LinearRegression       269694.38  387332.82  0.74  277373.69  397960.56  0.72
MLPRegressor           233633.22  355160.99  0.78  244526.68  370491.44  0.76
RandomForestRegressor  100626.54  171334.45  0.95  201132.37  333213.39  0.80
XGBRegressor            54590.70   79161.20  0.99  191111.21  317432.19  0.82

Após comparar os quatro algoritmos de regressão, o *XGBRegressor* foi escolhido como o melhor modelo para a previsão de preços. 
* Maior poder preditivo, explicando 84% da variação dos preços nos dados de teste (R²). 
* Apresentou o menor Erro Médio Absoluto (MAE) e o menor Erro Quadrático Médio (RMSE), tornando-o o mais preciso e confiável. 

In [47]:
# Salvando modelo para usar na aplicação
nome_arquivo_modelo = os.path.join("joblib", "modelo_xgb.joblib")
joblib.dump(model_xgb, nome_arquivo_modelo)

['joblib/modelo_xgb.joblib']